# 04_Evaluate.ipynb
### Evaluation notebook
Loads a trained model, runs predictions on the validation/test set, produces a classification report, confusion matrix, and saves figures to `outputs/figures/`.

In [ ]:
# Install required packages (run once)
!pip install tensorflow matplotlib seaborn scikit-learn opencv-python tqdm


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

plt.rcParams['figure.figsize'] = (8,6)


In [ ]:
# Parameters - edit if needed
MODEL_PATH = 'models/mobilenet_emotion.h5'  # change to your model path
VAL_DIR = 'data/cropped_faces/val'          # validation folder (flow_from_directory)
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32
OUT_DIR = 'outputs/figures'
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
print('Loading model...')
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()


In [ ]:
print('Preparing validation generator...')
datagen = ImageDataGenerator(rescale=1.0/255)
val_gen = datagen.flow_from_directory(
    VAL_DIR,
    target_size=INPUT_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
labels = list(val_gen.class_indices.keys())
print('Classes:', labels)


In [ ]:
print('Predicting on validation set...')
steps = int(np.ceil(val_gen.samples / val_gen.batch_size))
preds = model.predict(val_gen, steps=steps, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_gen.classes


In [ ]:
print('\nClassification Report:')
report = classification_report(y_true, y_pred, target_names=labels, digits=4)
print(report)
with open(os.path.join(OUT_DIR, 'classification_report.txt'), 'w') as f:
    f.write(report)


In [ ]:
print('Computing confusion matrix...')
cm = confusion_matrix(y_true, y_pred)
cm_path = os.path.join(OUT_DIR, 'confusion_matrix.png')
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig(cm_path, dpi=300)
plt.show()
print(f'Saved confusion matrix to: {cm_path}')


In [ ]:
acc = np.sum(y_pred == y_true) / len(y_true)
print(f'Validation Accuracy: {acc*100:.2f}%')
with open(os.path.join(OUT_DIR, 'summary.txt'), 'w') as f:
    f.write(f'Validation Accuracy: {acc*100:.2f}%\n')
    f.write('Confusion matrix saved to: ' + cm_path + '\n')
    f.write('Classification report saved to: ' + os.path.join(OUT_DIR, 'classification_report.txt') + '\n')


## Optional: Per-class precision/recall bar chart
This visualizes precision and recall for each class using the saved classification report.

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_true, y_pred, average=None, zero_division=0)
recall = recall_score(y_true, y_pred, average=None, zero_division=0)
x = np.arange(len(labels))
plt.figure(figsize=(10,5))
plt.bar(x - 0.15, precision, width=0.3, label='Precision')
plt.bar(x + 0.15, recall, width=0.3, label='Recall')
plt.xticks(x, labels, rotation=45)
plt.ylim(0,1)
plt.legend()
plt.title('Per-class Precision and Recall')
pr_path = os.path.join(OUT_DIR, 'precision_recall.png')
plt.tight_layout()
plt.savefig(pr_path, dpi=300)
plt.show()
print(f'Saved precision/recall plot to: {pr_path}')
